In [1]:
-- 2-1. Declaring Variables

USE AdventureWorks2014;

Commands completed successfully.

Total execution time: 00:00:00.003

In [2]:
DECLARE @AddressLine1 nvarchar(60) = 'Heiderplatz';
SELECT AddressID, AddressLine1
FROM Person.Address
WHERE AddressLine1 LIKE '%' + @AddressLine1 + '%';

(18 rows affected)

Total execution time: 00:00:00.116

AddressID,AddressLine1
20333,Heiderplatz 268
17062,Heiderplatz 268
24962,Heiderplatz 662
15742,Heiderplatz 662
27109,Heiderplatz 772
23496,Heiderplatz 772
27926,Heiderplatz 789
14368,Heiderplatz 918
21649,Heiderplatz 918
27516,Heiderplatz 918


In [3]:
-- 2-2. Retrieving a Value into a Variable

DECLARE @AddressLine1 nvarchar(60);
DECLARE @AddressLine2 nvarchar(60);
SELECT @AddressLine1 = AddressLine1, @AddressLine2 = AddressLine2
FROM Person.Address
WHERE AddressID = 66;
SELECT @AddressLine1 AS Address1, @AddressLine2 AS Address2

(1 row affected)

Total execution time: 00:00:00.009

Address1,Address2
4775 Kentucky Dr.,Unit E


In [4]:
-- 2-3. Writing an IF…THEN…ELSE Statement

DECLARE @QuerySelector int = 3;
IF @QuerySelector = 1
BEGIN
    SELECT TOP 3 ProductID, Name, Color
    FROM Production.Product
    WHERE Color = 'Silver'
    ORDER BY Name
END
ELSE
BEGIN
    SELECT TOP 3 ProductID, Name, Color
    FROM Production.Product
    WHERE Color = 'Black'
    ORDER BY Name
END;

(3 rows affected)

Total execution time: 00:00:00.013

ProductID,Name,Color
322,Chainring,Black
863,"Full-Finger Gloves, L",Black
862,"Full-Finger Gloves, M",Black


In [5]:
-- 2-4. Writing a Simple CASE Expression

SELECT DepartmentID AS DeptID, Name, GroupName,
    CASE GroupName
        WHEN 'Research and Development' THEN 'Room A'
        WHEN 'Sales and Marketing' THEN 'Room B'
        WHEN 'Manufacturing' THEN 'Room C'
    ELSE 'Room D'
    END AS ConfRoom
FROM HumanResources.Department

(16 rows affected)

Total execution time: 00:00:00.059

DeptID,Name,GroupName,ConfRoom
1,Engineering,Research and Development,Room A
2,Tool Design,Research and Development,Room A
3,Sales,Sales and Marketing,Room B
4,Marketing,Sales and Marketing,Room B
5,Purchasing,Inventory Management,Room D
6,Research and Development,Research and Development,Room A
7,Production,Manufacturing,Room C
8,Production Control,Manufacturing,Room C
9,Human Resources,Executive General and Administration,Room D
10,Finance,Executive General and Administration,Room D


In [6]:
-- 2-5. Writing a Searched CASE Expression

SELECT DepartmentID, Name,
    CASE
        WHEN Name = 'Research and Development' THEN 'Room A'
        WHEN (Name = 'Sales and Marketing' OR DepartmentID = 10) THEN 'Room B'
        WHEN Name LIKE 'T%'THEN 'Room C'
    ELSE 'Room D' END AS ConferenceRoom
FROM HumanResources.Department;

(16 rows affected)

Total execution time: 00:00:02.509

DepartmentID,Name,ConferenceRoom
12,Document Control,Room D
1,Engineering,Room D
16,Executive,Room D
14,Facilities and Maintenance,Room D
10,Finance,Room B
9,Human Resources,Room D
11,Information Services,Room D
4,Marketing,Room D
7,Production,Room D
8,Production Control,Room D


In [7]:
-- 2-6. Writing a WHILE Statement

-- Declare variables
DECLARE @AWTables TABLE (SchemaTable varchar(100));
DECLARE @TableName varchar(100);
-- Insert table names into the table variable
INSERT @AWTables (SchemaTable)
-- Report on each table using sp_spaceused
WHILE (SELECT COUNT(*) FROM @AWTables) > 0
BEGIN
    SELECT TOP 1 @TableName = SchemaTable
    FROM @AWTables
    ORDER BY SchemaTable;
    EXEC sp_spaceused @TableName;
    DELETE @AWTables
    WHERE SchemaTable = @TableName;
END;

: Msg 156, Level 15, State 1, Line 9
Incorrect syntax near the keyword 'WHILE'.

Total execution time: 00:00:00.003

In [8]:
-- 2-7. Returning from the Current Execution Scope

-- Solution #1: Exit with No Return Value

IF NOT EXISTS
    (SELECT ProductID
    FROM Production.Product
    WHERE Color = 'Pink')
BEGIN
    RETURN;
END;
SELECT ProductID
FROM Production.Product
WHERE Color = 'Pink';

Commands completed successfully.

Total execution time: 00:00:00.004

In [9]:
-- Solution #2: Exit and Provide a Value

CREATE PROCEDURE ReportPink AS
IF NOT EXISTS
(SELECT ProductID
FROM Production.Product
WHERE Color = 'Pink')
BEGIN
--Return the value 100 to indicate no pink products
RETURN 100;
END;
SELECT ProductID
FROM Production.Product
WHERE Color = 'Pink';
--Return the value 0 to indicate pink was found
RETURN 0;

: Msg 2714, Level 16, State 3, Procedure ReportPink, Line 3
There is already an object named 'ReportPink' in the database.

Total execution time: 00:00:00.003

In [10]:
DECLARE @ResultStatus int;
EXEC @ResultStatus = ReportPink;
PRINT @ResultStatus;

100

Total execution time: 00:00:00.004

In [11]:
-- 2-8. Going to a Label in a Transact-SQL Batch

DECLARE @Name nvarchar(50) = 'Engineering';
DECLARE @GroupName nvarchar(50) = 'Research and Development';
DECLARE @Exists bit = 0;
IF EXISTS (
    SELECT Name
    FROM HumanResources.Department
    WHERE Name = @Name)
BEGIN
    SET @Exists = 1;
    GOTO SkipInsert;
END;
INSERT INTO HumanResources.Department
    (Name, GroupName)
    VALUES(@Name , @GroupName);
SkipInsert: IF @Exists = 1
BEGIN
    PRINT @Name + ' already exists in HumanResources.Department';
END
ELSE
BEGIN
    PRINT 'Row added';
END;

Engineering already exists in HumanResources.Department

Total execution time: 00:00:00.004

In [12]:
-- 2-9. Pausing Execution for a Period of Time

-- How to delay for a specific number of hours:minutes:seconds
WAITFOR DELAY '00:00:10';
BEGIN
    SELECT TransactionID, Quantity
    FROM Production.TransactionHistory;
END;


(113443 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:10.657

TransactionID,Quantity
100000,2
100001,1
100002,1
100003,1
100004,1
100005,1
100006,1
100007,1
100008,1
100009,1


In [13]:
-- How to wait until a specific time is reached.
WAITFOR TIME '12:22:00';
BEGIN
    SELECT COUNT(*)
    FROM Production.TransactionHistory;
END;

(1 row affected)

Total execution time: 00:02:33.252

(No column name)
113443


In [14]:
-- 2-10. Creating and Using Cursors

-- Do not show rowcounts in the results
SET NOCOUNT ON;
DECLARE @session_id smallint;
-- Declare the cursor
DECLARE session_cursor CURSOR FORWARD_ONLY READ_ONLY FOR
    SELECT session_id
    FROM sys.dm_exec_requests
    WHERE status IN ('runnable', 'sleeping', 'running');
-- Open the cursor
OPEN session_cursor;
-- Retrieve one row at a time from the cursor
FETCH NEXT
    FROM session_cursor
    INTO @session_id;
-- Process and retrieve new rows until no more are available
WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT 'Spid #: ' + STR(@session_id);
    EXEC ('sp_who ' + @session_id);
    FETCH NEXT
        FROM session_cursor
        INTO @session_id;
END;
-- Close the cursor
CLOSE session_cursor;
-- Deallocate the cursor
DEALLOCATE session_cursor

Spid #: 11

Spid #: 12

Spid #: 18

Spid #: 19

Spid #: 20

...

spid,ecid,status,loginame,hostname,blk,dbname,cmd,request_id
39,0,sleeping,sa,,0,master,TASK MANAGER,0


spid,ecid,status,loginame,hostname,blk,dbname,cmd,request_id
40,0,sleeping,sa,,0,master,TASK MANAGER,0


spid,ecid,status,loginame,hostname,blk,dbname,cmd,request_id
41,0,sleeping,sa,,0,master,TASK MANAGER,0


spid,ecid,status,loginame,hostname,blk,dbname,cmd,request_id
42,0,sleeping,sa,,0,master,TASK MANAGER,0


spid,ecid,status,loginame,hostname,blk,dbname,cmd,request_id
54,0,runnable,AIR-INC\fwu,FWU4-LPT,0,AdventureWorks2014,SELECT,0
